In [1]:
import PIL
import tensorflow as tf
import numpy as np
from glob import glob
import time
import os

from matplotlib import pyplot as plt

IMG_H = 128
IMG_W = 128
IMG_C = 3  ## Change this to 1 for grayscale.


# Regularization Rate for each loss function

# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

# Loss function for evaluating adversarial loss
adv_loss_fn = tf.losses.MeanSquaredError()

w_init = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.02)

In [2]:
def load_image(image_path):
  img = tf.io.read_file(image_path)
  img = tf.io.decode_bmp(img)
  img = tf.image.resize_with_crop_or_pad(img, IMG_H, IMG_W)
  img = tf.cast(img, tf.float32)
  img = (img - 127.5) / 127.5
  return img

In [3]:
def tf_dataset(images_path, batch_size):
  dataset = tf.data.Dataset.from_tensor_slices(images_path)
  dataset = dataset.shuffle(buffer_size=10240)
  dataset = dataset.map(load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
  dataset = dataset.batch(batch_size)
  dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
  return dataset

In [4]:
def conv_block(input, num_filters):
    x = tf.keras.layers.Conv2D(num_filters, kernel_size=(1,1), padding="same")(input)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation("relu")(x)

    x = tf.keras.layers.Conv2D(num_filters, kernel_size=(3,3), padding="same")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation("relu")(x)

    return x

In [5]:
def decoder_block(input, skip_features, num_filters):
    x = tf.keras.layers.Conv2DTranspose(num_filters, (1, 1), strides=2, padding="same")(input)
    x = tf.keras.layers.Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

In [6]:
class ResUnetGAN(tf.keras.models.Model):
    def __init__(self, input_shape, batch_size):
        super(ResUnetGAN, self).__init__()
        self.discriminator = self.build_discriminator(input_shape)
        self.generator = self.build_generator_resnet50_unet(input_shape)
        self.batch_size = batch_size
        self.ADV_REG_RATE_LF = 1
        self.REC_REG_RATE_LF = 50
        self.SSIM_REG_RATE_LF = 50
        self.FEAT_REG_RATE_LF = 1
        self.d_optimizer = tf.keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5, beta_2=0.999)
        self.g_optimizer = tf.keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5, beta_2=0.999)

        checkpoint_dir = './training_checkpoints'
        self.checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
        self.checkpoint = tf.train.Checkpoint(generator_optimizer=self.g_optimizer,
                                 discriminator_optimizer=self.d_optimizer,
                                 generator=self.generator,
                                 discriminator=self.discriminator)
        # self.discriminator.summary()
        # self.generator.summary()

    # create generator model based on resnet50 and unet network
    def build_generator_resnet50_unet(self, input_shape):
        # print(inputs)
        # print("pretained start")
        """ Pre-trained ResNet50 Model """
        resnet50 = tf.keras.applications.ResNet50(include_top=False, weights="imagenet", input_tensor=input_shape)
        # print("testing")
        """ Encoder using resnet50"""
        # for layer in resnet50.layers:
        #   print(layer.name)
        s1 = resnet50.get_layer("input_1").output           ## (128 x 128)
        # print(s1)
        s2 = resnet50.get_layer("conv1_relu").output        ## (64 x 64)
        s3 = resnet50.get_layer("conv2_block3_out").output  ## (32 x 32)
        s4 = resnet50.get_layer("conv3_block4_out").output  ## (16 x 16)

        """ Bridge """
        b1 = resnet50.get_layer("conv4_block6_out").output  ## (32 x 32)

        # print("test")
        # print(b1.get_weights())
        """ Decoder unet"""
        d1 = decoder_block(b1, s4, 128)                     ## (16 x 16)
        d2 = decoder_block(d1, s3, 64)                     ## (32 x 32)
        d3 = decoder_block(d2, s2, 32)                     ## (64 x 64)
        d4 = decoder_block(d3, s1, 16)                      ## (128 x 128)

        """ Output """
        final_model = tf.keras.layers.Conv2D(3, 1, padding="same", activation="sigmoid")(d4)

        model = tf.keras.models.Model(inputs, outputs=[final_model, b1])

        return model
        # return outputs

    # create discriminator model

    def build_discriminator(self ,input_shape):

        x = tf.keras.layers.SeparableConvolution2D(32,kernel_size= (1, 1), strides=(2, 2), padding='same')(input_shape)
        x = tf.keras.layers.LeakyReLU()(x)
        x = tf.keras.layers.Dropout(0.3)(x)

        x = tf.keras.layers.SeparableConvolution2D(64,kernel_size=(1, 1), strides=(2, 2), padding='same')(x)
        x = tf.keras.layers.LeakyReLU()(x)
        x = tf.keras.layers.Dropout(0.3)(x)

        x = tf.keras.layers.Flatten()(x)
        x = tf.keras.layers.Dense(1)(x)

        model = tf.keras.models.Model(inputs, x)
        return model
        # return x

    def compile(self, d_optimizer, g_optimizer):
        super(ResUnetGAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer

  
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
    @tf.function
    def train_step(self, images):


        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            # tf.print("Images: ", images)
            reconstructed_images, low_feature = self.generator(images, training=True)
            real_output = self.discriminator(images, training=True)
            # print(generated_images.shape)
            fake_output = self.discriminator(reconstructed_images, training=True)
            
            # if tf.math.is_nan(real_output) is not None:
            #     tf.print(tf.math.is_nan(real_output)," real_output is NaN: ", real_output)
            # # else:
            #     # tf.print(real_output)
            # #
            # if tf.math.is_nan(fake_output) is not None:
            #     tf.print(tf.math.is_nan(fake_output)," fake_output is NaN: ", fake_output)
            # # else:
            # #     tf.print(fake_output)
            # #
            # if tf.math.is_nan(reconstructed_images) is not None:
            #     tf.print(tf.math.is_nan(reconstructed_images)," reconstructed_images is NaN: ",reconstructed_images)
            # else:
            #     tf.print(reconstructed_images)

            # Loss 1: ADVERSARIAL loss
            loss_adv = tf.reduce_mean(tf.math.log(real_output) + tf.math.log(1 - fake_output),keepdims=True)
            # Loss 2: RECONSTRUCTION loss
            loss_rec = tf.math.reduce_sum(tf.math.abs(images - reconstructed_images), keepdims=True)
            # Loss 3: SSIM loss
            loss_ssim = tf.math.reduce_sum(1 - tf.image.ssim(images,reconstructed_images, max_val=1.0)[0], keepdims=True)
            # Loss 4: FEATURE loss
            loss_feat = tf.math.reduce_sum(tf.math.square(real_output - fake_output), keepdims=True)

            gen_loss = tf.math.reduce_mean((loss_adv * self.ADV_REG_RATE_LF) + (loss_rec * self.REC_REG_RATE_LF) + (loss_ssim * self.SSIM_REG_RATE_LF) + (loss_feat * self.FEAT_REG_RATE_LF))
            disc_loss = tf.math.reduce_mean((loss_adv * self.ADV_REG_RATE_LF) + (loss_feat * self.FEAT_REG_RATE_LF))

        
        gradients_of_generator = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)
        
        self.g_optimizer.apply_gradients(zip(gradients_of_generator, self.generator.trainable_variables))
        self.d_optimizer.apply_gradients(zip(gradients_of_discriminator, self.discriminator.trainable_variables))

        # self.mean_loss_adv = tf.math.reduce_mean(loss_adv)
        # self.mean_loss_rec = tf.math.reduce_mean(loss_rec)
        # self.mean_loss_ssim = tf.math.reduce_mean(loss_ssim)
        # self.mean_loss_feat = tf.math.reduce_mean(loss_feat)

        # tf.summary.scalar('loss_adv', self.mean_loss_adv)
        # tf.summary.scalar('loss_rec', self.mean_loss_rec)
        # tf.summary.scalar('loss_ssim', self.mean_loss_ssim)
        # tf.summary.scalar('loss_feat', self.mean_loss_feat)
        # tf.summary.scalar('gen_loss', gen_loss)
        # tf.summary.scalar('disc_loss', disc_loss)


        return {
            "gen_loss": gen_loss,
            "disc_loss": disc_loss,
            "loss_adv": loss_adv,
            "loss_rec": loss_rec,
            "loss_ssim": loss_ssim,
            "loss_feat": loss_feat
        }

    def saved_model(self, filepath, num_of_epoch):
        self.generator.save(filepath + "g_model" + str(num_of_epoch) + ".h5")
        self.discriminator.save(filepath + "d_model" + str(num_of_epoch) + ".h5")

    def loaded_model(self, filepath):
        self.generator.load_weights(filepath)
        self.discriminator.load_weights(filepath)



    # def train(self, dataset, epochs):
    #     for epoch in range(epochs):
    #         start = time.time()
    #         print("Epoch: ",)
    #         for image_batch in dataset:
    #             # print(image_batch)
    #             output = self.train_step(image_batch)
    #             # tf.print(output)

    #         # Produce images for the GIF as you go
    #         # display.clear_output(wait=True)
    #         # generate_and_save_images(generator,
    #         #                      epoch + 1,
    #         #                      seed)
    #         # Save the model every 15 epochs
    #         if (epoch + 1) % 15 == 0:
    #             self.checkpoint.save(file_prefix = self.checkpoint_prefix)
    #             self.saved_model("/content/drive/MyDrive/mura_data/saved_model/", num_epochs)
    #             print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

In [7]:
if __name__ == "__main__":
    # run the function here
    print("start")
    ## Hyperparameters
    batch_size = 24
    input_shape = (IMG_W, IMG_H, IMG_C)
    # print(input_shape)

    """ Input """
    inputs = tf.keras.layers.Input(input_shape, name="input_1")

    num_epochs = 600
    train_images_path = glob("/content/drive/MyDrive/mura_data/train_data/*.bmp")


    # d_model = build_discriminator(inputs)
    # g_model = build_generator_resnet50_unet(inputs)

    resunetgan = ResUnetGAN(inputs, batch_size)


    g_optimizer = tf.keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5, beta_2=0.999)
    d_optimizer = tf.keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5, beta_2=0.999)
    resunetgan.compile(d_optimizer, g_optimizer)

    # print(train_images_path)
    train_images_dataset = tf_dataset(train_images_path, batch_size)

    # resunetgan.fit(train_images_dataset)

    # resunetgan.train(train_images_dataset, num_epochs)

    for epoch in range(num_epochs):
        print("Epoch: ", epoch)
        start = time.time()
        for image_batch in train_images_dataset:
        # print(image_batch.shape)
            # print("Images_batch: ", image_batch)
            resunetgan.fit(image_batch)
            resunetgan.saved_model("/content/drive/MyDrive/mura_data/saved_model/", num_epochs)

    # resunetgan.summary()
    # resunetgan.save_weights("saved_model/resunet_model")



start
Epoch:  0
1/1 [==============================] - 15s 15s/step - gen_loss: 41636584.0000 - disc_loss: -3.0850 - loss_adv: -3.1542 - loss_rec: 832730.6875 - loss_ssim: 1.0263 - loss_feat: 0.0692


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 58ms/step - gen_loss: 3850237.7500 - disc_loss: -2.8447 - loss_adv: -2.8585 - loss_rec: 77003.6016 - loss_ssim: 1.2141 - loss_feat: 0.0138
Epoch:  373
1/1 [==============================] - 0s 43ms/step - gen_loss: 3934795.2500 - disc_loss: -3.2552 - loss_adv: -3.2615 - loss_rec: 78694.8047 - loss_ssim: 1.1672 - loss_feat: 0.0064
Epoch:  374
1/1 [==============================] - 0s 53ms/step - gen_loss: nan - disc_loss: nan - loss_adv: nan - loss_rec: 79727.8828 - loss_ssim: 1.2121 - loss_feat: 0.0121
Epoch:  375
1/1 [==============================] - 0s 42ms/step - gen_loss: nan - disc_loss: nan - loss_adv: nan - loss_rec: 81337.4297 - loss_ssim: 1.2369 - loss_feat: 0.0027
Epoch:  376
1/1 [==============================] - 0s 49ms/step - gen_loss: nan - disc_loss: nan - loss_adv: nan - loss_rec: 80215.2969 - loss_ssim: 1.1898 - loss_feat: 0.0092
Epoch:  377
1/1 [==============================

In [ ]:
# load an image
def load_image_test(filename, size=(128,128)):
	# load image with the preferred size
	pixels = tf.keras.preprocessing.image.load_img(filename, target_size=size)
	# convert to numpy array
	pixels = tf.keras.preprocessing.image.img_to_array(pixels)
	# scale from [0,255] to [-1,1]
	pixels = (pixels - 127.5) / 127.5
	# reshape to 1 sample
	pixels = np.expand_dims(pixels, 0)
	return pixels

In [ ]:
# test_images_dataset = tf_dataset(test_images_path, batch_size)
# normal_images = glob('mura_data/mura_data/test_data/normal_*.bmp')
# defect_images = glob('mura_data/mura_data/test_data/defect_*.bmp')
# len_nor_data = len(normal_images)
# len_def_data = len(defect_images)
# print(len_nor_data)
# print(len_def_data)
# threshold = 0.6
# defect_preds = []
# for image in defect_images:
#   # print(image)
#   if "DS_Store" not in image:
#     src_image = load_image_test(image)
#
#     test = d_model.predict(src_image)
#     test = (test + 1) / 2.0
#     defect_preds = np.append(defect_preds,test)
#
#     # preds = (preds - preds.min())/(preds.max()-preds.min())
#     # print(test)
#
#
#
# normal_preds = []
# for image in normal_images:
#   # print(image)
#   if "DS_Store" not in image:
#     src_image = load_image_test(image)
#
#     test = d_model.predict(src_image)
#     test = (test + 1) / 2.0
#     normal_preds = np.append(normal_preds,test)
#
#     # preds = (preds - preds.min())/(preds.max()-preds.min())
#     # print(test)
#
#
# print(defect_preds)
# print(np.mean(defect_preds))
# true_def_pred = len(np.where(defect_preds > threshold)[0])
# print(true_def_pred)
#
#
# print(normal_preds)
# print(np.mean(normal_preds))
# true_nor_pred = len(np.where(normal_preds < threshold)[0])
# print(true_nor_pred)
#
# total_acc = (true_def_pred + true_nor_pred) / (len_nor_data + len_def_data) * 100
# print("total_accuracy: ", total_acc)



51
50
[0.64748341 0.64582115 0.64495695 0.63706183 0.64389598 0.64372158
 0.64029634 0.64423901 0.63870716 0.64056474 0.63585079 0.63704026
 0.6411283  0.64476216 0.63866848 0.63880849 0.63715732 0.63976002
 0.64145702 0.64556265 0.64365792 0.6378026  0.64362186 0.63863498
 0.64006096 0.64166403 0.64609551 0.63979173 0.64554846 0.63616109
 0.64451241 0.64017868 0.6469053  0.63884485 0.63761288 0.64289093
 0.64132708 0.64631164 0.64095742 0.63797343 0.64088923 0.63647813
 0.65117645 0.63967681 0.64133358 0.63783252 0.64289385 0.64198399
 0.63981831 0.63435316]
0.6412786686420441
50
[0.63939631 0.64166212 0.63711405 0.63674265 0.63656467 0.63669407
 0.63940752 0.63682628 0.63999265 0.63952291 0.63734055 0.64020377
 0.63860971 0.640733   0.64046967 0.63901877 0.63815737 0.63886672
 0.6363433  0.63823372 0.63949871 0.63427675 0.63712913 0.64116955
 0.63772613 0.63943613 0.63968712 0.6397562  0.63884401 0.64015615
 0.63704967 0.64029896 0.63756126 0.63938802 0.63810068 0.63947988
 0.6398171